Event                           Effect
update of revision attribute    sync
save cmd                        compare revisions |> persist delta revisions, replace modelview w section, sync
add partner_ref with id         create partner_ref,partner_ref_revision with id, persist, replace modelview w section, sync
delete partner_ref              delete/invalidate in db, replace modelview w section, sync
add product_item                create Product_item, persist, replace modelview, synchronize with client
delete product_item             delete/invalidate in db, replace modelview, synchronize with client
                

In [ ]:
push!(LOAD_PATH, "src")
using BitemporalPostgres, BitemporalReactive, JSON, LifeInsuranceDataModel, SearchLight, TimeZones, ToStruct
ENV["SEARCHLIGHT_USERNAME"] = "bitemporalpostgres"
ENV["SEARCHLIGHT_PASSWORD"] = "jw8s0F49KL"

model0 = JSON.parse(JSON.json(csection(1, now(tz"UTC"), now(tz"UTC"))))
model1 = JSON.parse(JSON.json(csection(1, now(tz"UTC"), now(tz"UTC"))))



In [ ]:
model1["revision"]["description"] = "first mutation by GUI model"
model1["revision"]["id"]["value"] = nothing
model1["revision"]["description"] = "CR first mutation by GUI model"
model1["partner_refs"][1]["rev"]["description"] = "CPR first mutation by GUI model"
model1["product_items"][1]["revision"]["description"] = "PIR first mutation by GUI model"
model1["product_items"][1]["tariff_items"][1]["tariff_ref"]["rev"]["defement"] = 9
model1["product_items"][1]["tariff_items"][1]["tariff_ref"]["rev"]["description"] = "TIR first mutation by GUI model"
model1["product_items"][1]["tariff_items"][1]["partner_refs"][1]["rev"]["description"] = "bubu|"

ToStruct.tostruct(ContractRevision, model1["revision"])


In [ ]:

changed = ToStruct.tostruct(ContractSection, model1)
@test(changed.revision.description == "CR first mutation by GUI model")
@test(changed.partner_refs[1].rev.description == "CPR first mutation by GUI model")
@test(changed.product_items[1].revision.description == "PIR first mutation by GUI model")
@test(changed.product_items[1].tariff_items[1].tariff_ref.rev.deferment == 9)
@test(changed.product_items[1].tariff_items[1].tariff_ref.rev.description == "TIR first mutation by GUI model")
@test(changed.product_items[1].tariff_items[1].partner_refs[1].rev.description == "bubu|")
BitemporalReactive.compareModelStateContract(model0, model1)